In [ ]:
import sys
sys.path.insert(0, "../input/fpo-hubmap-github-dl/Kaggle-HuBMap-Unet")

# Compute a Test Set single image

We'll compute the inference of a single image by :
* Patching the image with 2048x2048px patches
* Rescaling patches from 2048x2048px to 512x512px
* Applying a threshold of 0.6 to the output of the softmax function second layer
* Applying an Errosion / Dilatation to remove outliers pixels.

In [ ]:
from core.models.UNet_Inference import UNet_Inference
from core.ImUtils import mask2rle,read_mask_file
from skimage.io import imread,imsave
IMAGE_PATH="../input/hubmap-kidney-segmentation/test/b2dc8411c.tiff"

MODEL_PATH="../input/fpo-hubmap-github-dl/Kaggle-HuBMap-Unet/checkpoints/CP_2048_0.25_epoch63_loss0.013.pth"
TILESIZE=2048
DOWNSAMPLING=0.25
COVERING=0.5
BatchSize=20
threshold=0.6
kernelsize=7
device='cuda:0'
model=UNet_Inference(nclasses=2)
model.LoadCheckpoint(MODEL_PATH)

prediction=model.inferImage(IMAGE_PATH,
                            threshold=threshold,
                            kernelsize=kernelsize,
                            TileSize=TILESIZE,
                            TileCovering=COVERING,
                            DownScaling=DOWNSAMPLING,
                            BatchSize=BatchSize,
                            inferdevice=device,mode='half')

imsave('prediction.tiff',prediction)
del prediction

We'll then loop over image in order to find the 100 first positive images...
To do so, we'll loop over indexes until we  have 100 positive element with more than 100x100px NC to display.

In this part, we'll use directly the TileManager class in order to explode the image in tiles.
This class has just to be innitialized to be use to transform / inverse transform an image to/from patch.

In [ ]:
from core.TileManager import TileManager
from core.ImUtils import read_tiff_file
import matplotlib.pyplot as plt
import numpy as np

max_images=100 # imgs max
Size_Target=100 # px

prediction=imread('prediction.tiff')>0
img=read_tiff_file(IMAGE_PATH)
prediction=np.expand_dims(prediction,axis=-1).astype(np.bool)
TM=TileManager(TILESIZE,COVERING,DOWNSAMPLING)
TM.fit(img.shape)

nb_positive=0
for i in range(len(TM)):
    if nb_positive>max_images:
        break;
    else:
        target=TM.transform(prediction,i)
        if (target>0).sum()>Size_Target*Size_Target:
            reference=TM.transform(img,i)
            fig,ax=plt.subplots(1,2,figsize=(10,5))
            ax[0].imshow(reference)
            ax[0].set_title('Reference')
            ax[0].axis('off')
            ax[1].imshow(target.squeeze())
            ax[1].set_title('Detected')
            ax[1].axis('off')
            plt.show()
            nb_positive+=1
            